In [1]:
using DelimitedFiles

include("energy_loss/energy_loss.jl")
include("proton_flux/proton_flux.jl")
include("particle_propagation/numerical_propagation.jl")

E_g (generic function with 1 method)

# Interações com fótons de fundo

### Produção de pares

In [4]:
xi = range(0.4, 4, 100)
xi = 10 .^(xi)

phi_xi = phi.(xi)

open("outputs/phi_xi.txt", "w") do io
    writedlm(io, ["xi" "phi_xi"])
    writedlm(io, [xi phi_xi])
end

In [5]:
nu = range(-3, 2, 100)
nu = 10 .^ (nu)

fnu = f_nu.(nu)

open("outputs/f_nu.txt", "w") do io
    writedlm(io, ["nu" "f_nu"])
    writedlm(io, [nu fnu])
end

### Todas as interações

In [2]:
E = range(17, 22, 1000)
E = 10 .^ (E)

beta_pair_E = beta_pair.(E)
beta_pion_E = beta_pion.(E)
beta_rsh_E = fill(beta_rsh(), 1000)
beta_total_E = beta.(E, 0)

open("outputs/beta_E.txt", "w") do io
    writedlm(io, ["E" "beta_pair_E" "beta_pion_E" "beta_rsh_E" "beta_total_E"])
    writedlm(io, [E beta_pair_E beta_pion_E beta_rsh_E beta_total_E])
end

In [5]:
E = range(17, 22, 1000)
E = 10 .^ (E)

db0_pair_dE_E = db0_pair_dE.(E)
db0_pion_dE_E = db0_pion_dE.(E)
db0_rsh_dE_E = fill(db0_rsh_dE(), 1000)
db0_dE_E = db0_dE.(E)

open("outputs/dbeta_dE_E.txt", "w") do io
    writedlm(io, ["E" "db0_pair_dE" "db0_pion_dE" "db0_rsh_dE" "db0_dE"])
    writedlm(io, [E db0_pair_dE_E db0_pion_dE_E db0_rsh_dE_E db0_dE_E])
end

# Propagação de partículas

Resolve a equação

$$
\frac{dE}{dt} = -E \frac{dt}{dz}\beta(E, z)
$$

utilizando de condições iniciais $z = z_{src}$ e $E = E_{src}$ até $z = 0$.

In [7]:
include("particle_propagation/numerical_propagation.jl")

E = [1e18 1e19 1e20 1e21]

for (i, E_src) in enumerate(E)
    
    open("outputs/one_particle_propagation/E_t_$i.txt", "w") do io

        x, y = proton_propa(E_src, 0.4)

        writedlm(io, ["z" "E"])
        writedlm(io, [x y])
            
    end
end

In [8]:
include("particle_propagation/numerical_propagation.jl")

E = [1e18 1e19 1e20 1e21]

for (i, E_src) in enumerate(E)
    
    open("outputs/one_particle_propagation/E_t_$i.txt", "w") do io

        x, y = proton_propa(E_src, 0.4)

        writedlm(io, ["z" "E"])
        writedlm(io, [x y])
            
    end
end

A partir da equação acima, utilizando de condições iniciais $z = 0$ e $E = E$, é mapeado a energia de geração $E_g(E, z_g)$ até $z = z_{src}$.

In [10]:
include("particle_propagation/numerical_propagation.jl")

z_srcs = [0.01 0.05 0.5 1 1.5]

E = range(17, 21, 1000)
E = 10 .^(E)

for (i, z_src) in enumerate(z_srcs)
    
    open("outputs/generation_energy/E_g_$i.txt", "w") do io

        Eg = E_g.(E, z_src)

        writedlm(io, ["E" "Eg"])
        writedlm(io, [E Eg])
            
    end
end

Calcula a integral de mapeamento de um intervalo de energia da fonte com a Terra

$$
\frac{dE_g}{dE} = (1 + z_g)\exp{\left[\frac{1}{H_0}\int^{z_g}_0dz(1 + z)^{1/2}\left(\frac{db_0(E)}{dE}\right)_{E = E_g(z)(1 + z)}\right]}
$$

In [3]:
include("proton_flux/proton_flux.jl")

z_srcs = [7.7E-3 0.04 8.5e-2]

E = range(17, 21, 50)
E = 10 .^(E)

for (i, z) in enumerate(z_srcs)
    
    open("outputs/energy_connection/dEsrc_dE_$i.txt", "w") do io
        
        dEsrc_dE_z = dEg_dE.(z, E)

        writedlm(io, ["E" "dEsrc_dE"])
        writedlm(io, [E dEsrc_dE_z])
    end
end 


# Fator de modificação

In [15]:
include("proton_flux/proton_flux.jl")

E = range(17, 21, 500)
E = 10 .^ (E)

z_srcs = [2.3e-3 3.8e-3 7.7e-3 2.4e-2 4E-2 8.5E-2 0.19 0.33 0.51 0.76 1.17]

for (i, z_mod) in enumerate(z_srcs)
    
    open("outputs/modification_factor/mod_factor_E_$i.txt", "w") do io

        modification_factor = mod_factor.(E, z_mod)
        
        writedlm(io, ["E" "mod_factor"])
        writedlm(io, [E modification_factor])
    end
end

Resolvendo para muitas fontes

In [ ]:
include("proton_flux/proton_flux_many_sources.jl")

E = range(17, 21, 500)
E = 10 .^ (E)

z_max = [0 3.8e-3 7.7e-3 2.4e-2 4E-2 8.5E-2 0.19 0.33 0.51 0.76 1.17]

for i = 6:11

    open("outputs/mod_factor_ms/mod_factor_ms_E_$(i - 1).txt", "w") do io

        mod_factor_ms = mod_factor_many_sources.(E, z_max[i], z_max[i - 1])
                
        writedlm(io, z_max[i])
        writedlm(io, ["E" "mod_factor_ms"])
        writedlm(io, [E mod_factor_ms])
    end
end

E = 1.0e17
E = 1.0186289902446845e17
E = 1.037605019766914e17
E = 1.0569345535579874e17
E = 1.076624177045489e17
E = 1.0966805983368696e17
E = 1.1171106505048219e17
E = 1.1379212939153186e17
E = 1.1591196185988858e17
E = 1.1807128466661867e17
E = 1.2027083347685144e17
E = 1.2251135766041178e17
E = 1.2479362054713062e17
E = 1.27118399686903e17
E = 1.2948648711459022e17
E = 1.3189868961986746e17
E = 1.3435582902208262e17
E = 1.3685874245025149e17
E = 1.3940828262825814e17
E = 1.4200531816536818e17
E = 1.4465073385216413e17
E = 1.4734543096198374e17
E = 1.5009032755797334e17
E = 1.5288635880587357e17
E = 1.557344772926135e17
E = 1.5863565335085862e17
E = 1.6159087538959226e17
E = 1.6460115023085498e17
E = 1.676675034527694e17
E = 1.7079097973894304e17
E = 1.739726432343799e17
E = 1.7721357790803658e17
E = 1.80514887922111e17
E = 1.8387769800823232e17
E = 1.8730315385064422e17
E = 1.907924224765265e17
E = 1.9434669265360144e17
E = 1.9796717529513366e17
E = 2.0165510387247443e17
E = 2.05411

sys:1: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.


4.180637241455762e19
E = 4.258518291843406e19
E = 4.337850187559002e19
E = 4.418659956385941e19
E = 4.500975129608033e19
E = 4.584823751389105e19
E = 4.670234388327331e19
E = 4.75723613918787e19
E = 4.8458586448165e19
E = 4.936132098237907e19
E = 5.0280872549424955e19
E = 5.121755443364241e19
E = 5.217168575554333e19
E = 5.314359158053252e19
E = 5.413360302965376e19
E = 5.514205739240279e19
E = 5.6169298241638154e19
E = 5.72156755506324e19
E = 5.828154581230864e19
E = 5.9367272160691266e19
E = 6.047322449462631e19
E = 6.159977960380181e19
E = 6.274732129711574e19
E = 6.391624053343979e19
E = 6.510693555481468e19
E = 6.631981202212662e19
E = 6.7555283153316676e19
E = 6.88137698641567e19
E = 7.0095700911656034e19
E = 7.140151304013418e19
E = 7.273165113001454e19
E = 7.408656834939537e19
E = 7.546672630843902e19
E = 7.68725952166372e19
E = 7.830465404301215e19
E = 7.97633906792928e19
E = 8.124930210614031e19
E = 8.27628945624637e19
E = 8.430468371788967e19
E = 8.587519484845144e19
E = 8.7